In [1]:
%load_ext autoreload
%autoreload 2

from kaggle_speech_recog import *
import numpy as np
import xgboost as xgb

# Train

## Get feature and label matrices

In [2]:
X_vector_len = 16000
train = SpeechList.get_train('train/audio')
XY_train_valid = train.get_group_unknown_spectrogram_X_and_Y(X_vector_len=X_vector_len, spec_v='3', take_log=False, 
                                                             split_noise=True, n_fabricate_noise=1600, 
                                                             split=0.85)

SKIPPED train/audio/_background_noise_/README.md File format b'# Ba'... not understood.


/usr/local/anaconda3/lib/python3.5/site-packages/scipy/io/wavfile.py:273: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)
/usr/local/anaconda3/lib/python3.5/site-packages/librosa/filters.py:261: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  warnings.warn('Empty filters detected in mel frequency basis. '


In [3]:
random_pick_unknown = True

X_train_known, X_train_unknown, Y_train_known, Y_train_unknown, X_valid, Y_valid = XY_train_valid

if random_pick_unknown:
    i_train_unknown = np.random.choice(len(X_train_unknown), size=int(len(X_train_known) / 11), replace=False)
    random_X_train_unknown = X_train_unknown[i_train_unknown, :]
    random_Y_train_unknown = Y_train_unknown[i_train_unknown, :]

    X_train = np.concatenate((X_train_known, random_X_train_unknown))
    Y_train = np.concatenate((Y_train_known, random_Y_train_unknown))
else:
    X_train = np.concatenate((X_train_known, X_train_unknown))
    Y_train = np.concatenate((Y_train_known, Y_train_unknown))
    
Y_train_argmax = np.argmax(Y_train, axis=1)
Y_valid_argmax = np.argmax(Y_valid, axis=1)

## Get logits via each model

In [4]:
logs = ['SpectrogramMultiLSTMRandomInputModify_graph_03_lr_decay_run_01_group_unknown.log', 
        'SpectrogramMultiLSTMRandomInputModify2_graph_02_lower_lr_lr_decay_run_01_group_unknown.log',
        'SpectrogramConcatCLSTM_graph_04_conv5x1_lower_lr_run_01_group_unknown.log',
        'SpectrogramConcatCLSTM_graph_03_conv3x3_lower_lr_run_01_group_unknown.log',
        'SpectrogramConcatCLSTM_graph_02_lower_lr_run_01_group_unknown.log']

L_train = gather_logits(X_train, log_dir='logs', logs=logs)
L_valid = gather_logits(X_valid, log_dir='logs', logs=logs)

INFO:tensorflow:Restoring parameters from checkpoints/SpectrogramMultiLSTMRandomInputModify_graph_03_lr_decay_run_01_group_unknown/best/model-57354
INFO:tensorflow:Restoring parameters from checkpoints/SpectrogramMultiLSTMRandomInputModify2_graph_02_lower_lr_lr_decay_run_01_group_unknown/best/model-53088
INFO:tensorflow:Restoring parameters from checkpoints/SpectrogramConcatCLSTM_graph_04_conv5x1_lower_lr_run_01_group_unknown/best/model-229416
INFO:tensorflow:Restoring parameters from checkpoints/SpectrogramConcatCLSTM_graph_03_conv3x3_lower_lr_run_01_group_unknown/best/model-142358
INFO:tensorflow:Restoring parameters from checkpoints/SpectrogramConcatCLSTM_graph_02_lower_lr_run_01_group_unknown/best/model-53167
INFO:tensorflow:Restoring parameters from checkpoints/SpectrogramMultiLSTMRandomInputModify_graph_03_lr_decay_run_01_group_unknown/best/model-57354
INFO:tensorflow:Restoring parameters from checkpoints/SpectrogramMultiLSTMRandomInputModify2_graph_02_lower_lr_lr_decay_run_01_gr

## Train

In [5]:
xgb_train = xgb.DMatrix(L_train, label=Y_train_argmax)
xgb_valid = xgb.DMatrix(L_valid, label=Y_valid_argmax)

params = {'objective': 'multi:softprob',  # labels are 1-dim, not one-hot encoded
          'num_class': Y_train.shape[-1],
          'eval_metric': 'mlogloss', 
          'eta': 0.05,
          'subsample': 0.5,
          'max_depth': 1, 
          'colsample_bytree': 0.5}
watchlist = [(xgb_train, 'train'), (xgb_valid, 'valid')]

xgboost = xgb.train(params, xgb_train, num_boost_round=1000, evals=watchlist, 
                    early_stopping_rounds=50, verbose_eval=10)

# logs = ['SpectrogramMultiLSTMRandomInputModify_graph_03_lr_decay_run_01_group_unknown.log', 
#         'SpectrogramConcatCLSTM_graph_02_lower_lr_run_01_group_unknown.log']

# all_unknown, eta=0.05, gamma=0.1, ll=0.184892
# random_unknown, eta=0.05, gamma=0.1, ll=0.115045
# random_unknown, eta=0.05, gamma=0, ll=0.114929  * submitted, 01, LB 82%
# random_unknown, eta=0.05, subsample=0.5, ll=0.103648
# random_unknown, eta=0.05, subsample=0.5, lambda=0.1, ll=0.107599

# random_unknown, eta=0.05, subsample=0.5, max_depth=4, ll=0.102777
# random_unknown, eta=0.05, subsample=0.5, max_depth=3, ll=0.100699
# random_unknown, eta=0.05, subsample=0.5, max_depth=2, ll=0.09705
# random_unknown, eta=0.05, subsample=0.5, max_depth=1, ll=0.092102

# random_unknown, eta=0.05, subsample=0.5, max_depth=1, colsample_bytree=0.5, ll=0.085078
# random_unknown, eta=0.05, subsample=0.5, max_depth=3, colsample_bytree=0.5, ll=0.09212

# all_unknown, eta=0.05, subsample=0.5, max_depth=1, colsample_bytree=0.5, ll=0.142411
# all_unknown, eta=0.05, subsample=1, max_depth=1, colsample_bytree=0.5, ll=0.144147

# random_unknown, eta=0.05, subsample=0.5, max_depth=1, colsample_bytree=0.5, min_child_weight=10, ll=0.086196
# random_unknown, eta=0.05, subsample=0.5, max_depth=1, colsample_bytree=0.5, min_child_weight=5, ll=0.08582
# random_unknown, eta=0.05, subsample=0.5, max_depth=1, colsample_bytree=0.5, min_child_weight=3, ll=0.085768
# random_unknown, eta=0.05, subsample=0.5, max_depth=1, colsample_bytree=0.5, min_child_weight=2, ll=0.085757

#logs = ['SpectrogramMultiLSTMRandomInputModify_graph_03_lr_decay_run_01_group_unknown.log', 
#        'SpectrogramMultiLSTMRandomInputModify2_graph_02_lower_lr_lr_decay_run_01_group_unknown.log',
#        'SpectrogramConcatCLSTM_graph_04_conv5x1_lower_lr_run_01_group_unknown.log',
#        'SpectrogramConcatCLSTM_graph_03_conv3x3_lower_lr_run_01_group_unknown.log',
#        'SpectrogramConcatCLSTM_graph_02_lower_lr_run_01_group_unknown.log']

# random_unknown, eta=0.1, subsample=0.5, max_depth=1, colsample_bytree=0.5, ll=0.073678
# random_unknown, eta=0.05, subsample=0.5, max_depth=1, colsample_bytree=0.5, ll=0.072128 * submitted, 02, LB 82%
# random_unknown, eta=0.01, subsample=0.5, max_depth=1, colsample_bytree=0.5, ll=0.073888

[0]	train-mlogloss:2.19831	valid-mlogloss:2.20288
Multiple eval metrics have been passed: 'valid-mlogloss' will be used for early stopping.

Will train until valid-mlogloss hasn't improved in 50 rounds.
[10]	train-mlogloss:1.05511	valid-mlogloss:1.07874
[20]	train-mlogloss:0.616678	valid-mlogloss:0.646452
[30]	train-mlogloss:0.38314	valid-mlogloss:0.416269
[40]	train-mlogloss:0.246256	valid-mlogloss:0.281491
[50]	train-mlogloss:0.163913	valid-mlogloss:0.199933
[60]	train-mlogloss:0.113639	valid-mlogloss:0.150318
[70]	train-mlogloss:0.082338	valid-mlogloss:0.119863
[80]	train-mlogloss:0.062707	valid-mlogloss:0.10097
[90]	train-mlogloss:0.050024	valid-mlogloss:0.089939
[100]	train-mlogloss:0.041411	valid-mlogloss:0.082802
[110]	train-mlogloss:0.035462	valid-mlogloss:0.078658
[120]	train-mlogloss:0.031199	valid-mlogloss:0.07599
[130]	train-mlogloss:0.028044	valid-mlogloss:0.074585
[140]	train-mlogloss:0.025512	valid-mlogloss:0.074135
[150]	train-mlogloss:0.023567	valid-mlogloss:0.073974
[

# Predict

## Get feature matrix

In [6]:
X_vector_len = 16000
test = SpeechList.get_test('test/audio')
X_test = test.get_spectrogram_feature_ndarray(vector_len=X_vector_len, spec_v='3', take_log=False)

/usr/local/anaconda3/lib/python3.5/site-packages/librosa/filters.py:261: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  warnings.warn('Empty filters detected in mel frequency basis. '


## Get logits via each model

In [7]:
L_test = gather_logits(X_test, log_dir='logs', logs=logs)  # logs from above

INFO:tensorflow:Restoring parameters from checkpoints/SpectrogramMultiLSTMRandomInputModify_graph_03_lr_decay_run_01_group_unknown/best/model-57354
INFO:tensorflow:Restoring parameters from checkpoints/SpectrogramMultiLSTMRandomInputModify2_graph_02_lower_lr_lr_decay_run_01_group_unknown/best/model-53088
INFO:tensorflow:Restoring parameters from checkpoints/SpectrogramConcatCLSTM_graph_04_conv5x1_lower_lr_run_01_group_unknown/best/model-229416
INFO:tensorflow:Restoring parameters from checkpoints/SpectrogramConcatCLSTM_graph_03_conv3x3_lower_lr_run_01_group_unknown/best/model-142358
INFO:tensorflow:Restoring parameters from checkpoints/SpectrogramConcatCLSTM_graph_02_lower_lr_run_01_group_unknown/best/model-53167


## Predict

In [8]:
xgb_test = xgb.DMatrix(L_test)
Y_test = xgboost.predict(xgb_test, ntree_limit=xgboost.best_ntree_limit)  # ntree_limit is from above, best valid ll
test.add_predicted_label(Y_test, le=train.le)
test.save_submission_csv(dir_='submissions', name='EnsembleXGBoost_02_five_models.csv')

In [9]:
test[0].hear_and_see()

None, test/audio/clip_c24125cd1.wav, sample rate 16000, data length 16000
prediction: stop


# Save model

In [10]:
xgboost.save_model('ensemble_xgboost_model')

In [11]:
xgboost.best_ntree_limit

156